In [ ]:
!pip install torch
!pip install torchprofile

In [ ]:
import torch
import time
import psutil
from prettytable import PrettyTable
from torchprofile import profile_macs
from sklearn.metrics import precision_score, recall_score, f1_score
import torchvision.models.video as models
import warnings

# Подавление ошибок torchprofile
warnings.filterwarnings("ignore", category=UserWarning, module="torchprofile")

# Проверка наличия GPU
use_gpu = torch.cuda.is_available()

# Фиксируем random seed для воспроизводимости
torch.manual_seed(42)

# Модель X3D_XS
model_x3d_xs = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs')

# Модель Slow R50
model_slow_r50 = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50')

# Создание тестовых входных данных для X3D_XS
input_data_x3d_xs = torch.randn(1, 3, 32, 256, 256)
input_data_slow_r50 = torch.randn(1, 3, 32, 256, 256)

# Перемещение моделей на устройство (GPU, если доступен)
device = torch.device("cuda" if use_gpu else "cpu")
model_x3d_xs = model_x3d_xs.to(device)
model_slow_r50 = model_slow_r50.to(device)
input_data_x3d_xs = input_data_x3d_xs.to(device)
input_data_slow_r50 = input_data_slow_r50.to(device)

# Установка моделей в режим оценки
model_x3d_xs.eval()
model_slow_r50.eval()

# Измерение времени выполнения и потребления памяти для X3D_XS
start_time = time.time()
with torch.no_grad():
    output_x3d_xs = model_x3d_xs(input_data_x3d_xs)
end_time = time.time()
elapsed_time_x3d_xs = end_time - start_time
memory_allocated_x3d_xs = torch.cuda.max_memory_allocated(device) / 1e9 if use_gpu else 0

# Измерение FLOPs и параметров для X3D_XS
flops_x3d_xs = profile_macs(model_x3d_xs, (input_data_x3d_xs,))
flops_x3d_xs = f"{flops_x3d_xs / 1e9:.3f} G"
params_x3d_xs = sum(p.numel() for p in model_x3d_xs.parameters()) / 1e6
params_x3d_xs = f"{params_x3d_xs:.3f} M"

# Измерение времени выполнения и потребления памяти для Slow R50
start_time = time.time()
with torch.no_grad():
    output_slow_r50 = model_slow_r50(input_data_slow_r50)
end_time = time.time()
elapsed_time_slow_r50 = end_time - start_time
memory_allocated_slow_r50 = torch.cuda.max_memory_allocated(device) / 1e9 if use_gpu else 0

# Измерение FLOPs и параметров для Slow R50
flops_slow_r50 = profile_macs(model_slow_r50, (input_data_slow_r50,))
flops_slow_r50 = f"{flops_slow_r50 / 1e9:.3f} G"
params_slow_r50 = sum(p.numel() for p in model_slow_r50.parameters()) / 1e6
params_slow_r50 = f"{params_slow_r50:.3f} M"

# Расчет производительности GPU (FPS) и пропускной способности только если GPU доступен
if use_gpu:
    fps_x3d_xs = 1 / elapsed_time_x3d_xs
    fps_slow_r50 = 1 / elapsed_time_slow_r50
    throughput_x3d_xs = input_data_x3d_xs.size(2) / elapsed_time_x3d_xs
    throughput_slow_r50 = input_data_slow_r50.size(2) / elapsed_time_slow_r50
else:
    fps_x3d_xs, fps_slow_r50, throughput_x3d_xs, throughput_slow_r50 = 0, 0, 0, 0

# Расчет использования GPU (если доступен)
gpu_percent_x3d_xs = memory_allocated_x3d_xs / torch.cuda.max_memory_allocated(device) * 100 if use_gpu else 0
gpu_percent_slow_r50 = memory_allocated_slow_r50 / torch.cuda.max_memory_allocated(device) * 100 if use_gpu else 0

# Расчет использования CPU
cpu_percent = psutil.cpu_percent()

# Создание таблицы метрик
table = PrettyTable()
table.field_names = ["Метрика", "X3D_XS", "Slow_R50"]
table.add_row(["Время выполнения (сек)", f"{elapsed_time_x3d_xs:.4f}", f"{elapsed_time_slow_r50:.4f}"])
table.add_row(["Использование памяти GPU (ГБ)", f"{memory_allocated_x3d_xs:.4f}", f"{memory_allocated_slow_r50:.4f}"])
table.add_row(["FLOPs", flops_x3d_xs, flops_slow_r50])
table.add_row(["Параметры (М)", params_x3d_xs, params_slow_r50])
table.add_row(["Производительность GPU (FPS)", f"{fps_x3d_xs:.2f}", f"{fps_slow_r50:.2f}"])
table.add_row(["Пропускная способность (кадров/сек)", f"{throughput_x3d_xs:.2f}", f"{throughput_slow_r50:.2f}"])
table.add_row(["Использование GPU (%)", f"{gpu_percent_x3d_xs:.2f}", f"{gpu_percent_slow_r50:.2f}"])
table.add_row(["Использование CPU (%)", f"{cpu_percent:.2f}", f"{cpu_percent:.2f}"])

# Вывод таблицы
print(table)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


+-------------------------------------+----------+-----------+
|               Метрика               |  X3D_XS  |  Slow_R50 |
+-------------------------------------+----------+-----------+
|        Время выполнения (сек)       |  5.6104  |  13.8337  |
|    Использование памяти GPU (ГБ)    |  0.0000  |   0.0000  |
|                FLOPs                | 13.401 G | 218.617 G |
|            Параметры (М)            | 3.794 M  |  32.454 M |
|     Производительность GPU (FPS)    |   0.00   |    0.00   |
| Пропускная способность (кадров/сек) |   0.00   |    0.00   |
|        Использование GPU (%)        |   0.00   |    0.00   |
|        Использование CPU (%)        |  42.70   |   42.70   |
+-------------------------------------+----------+-----------+


In [ ]:
import torch
import time
import psutil
from prettytable import PrettyTable
from torchprofile import profile_macs
import torchvision.models.video as models
import warnings
from sklearn.metrics import accuracy_score, f1_score

# Подавление ошибок torchprofile
warnings.filterwarnings("ignore", category=UserWarning, module="torchprofile")

# Фиксируем random seed для воспроизводимости
torch.manual_seed(42)

# Проверка наличия GPU
use_gpu = torch.cuda.is_available()

# Модель X3D_XS
model_x3d_xs = torch.hub.load('facebookresearch/pytorchvideo', 'x3d_xs')

# Модель Slow R50
model_slow_r50 = torch.hub.load('facebookresearch/pytorchvideo', 'slow_r50')

# Создание обучающих входных данных для X3D_XS
train_input_data_x3d_xs = torch.randn(1, 3, 32, 256, 256)
train_input_data_slow_r50 = torch.randn(1, 3, 32, 256, 256)

# Перемещение моделей на устройство (GPU, если доступен)
device = torch.device("cuda" if use_gpu else "cpu")
model_x3d_xs = model_x3d_xs.to(device)
model_slow_r50 = model_slow_r50.to(device)
train_input_data_x3d_xs = train_input_data_x3d_xs.to(device)
train_input_data_slow_r50 = train_input_data_slow_r50.to(device)

# Установка моделей в режим обучения
model_x3d_xs.train()
model_slow_r50.train()

# Функция потерь
criterion = torch.nn.CrossEntropyLoss()

# Задаём оптимизатор
optimizer_x3d_xs = torch.optim.SGD(model_x3d_xs.parameters(), lr=0.001)
optimizer_slow_r50 = torch.optim.SGD(model_slow_r50.parameters(), lr=0.001)

# Количество эпох обучения
num_epochs = 2

# Создание таблицы метрик
table = PrettyTable()
table.field_names = ["Метрика", "X3D_XS", "Slow_R50"]

# Количество классов
num_classes = 2

for epoch in range(num_epochs):
    # Обучение модели X3D_XS
    start_time = time.time()
    optimizer_x3d_xs.zero_grad()
    output_x3d_xs = model_x3d_xs(train_input_data_x3d_xs)

    # Генерация случайных меток
    target_x3d_xs = torch.randint(0, num_classes, (1,), dtype=torch.long).to(device)

    loss_x3d_xs = criterion(output_x3d_xs, target_x3d_xs)
    loss_x3d_xs.backward()
    optimizer_x3d_xs.step()
    end_time = time.time()
    elapsed_time_x3d_xs = end_time - start_time

    # Обучение модели Slow R50
    start_time = time.time()
    optimizer_slow_r50.zero_grad()
    output_slow_r50 = model_slow_r50(train_input_data_slow_r50)

    # Генерация случайных меток для Slow R50
    target_slow_r50 = torch.randint(0, num_classes, (1,), dtype=torch.long).to(device)

    loss_slow_r50 = criterion(output_slow_r50, target_slow_r50)
    loss_slow_r50.backward()
    optimizer_slow_r50.step()
    end_time = time.time()
    elapsed_time_slow_r50 = end_time - start_time

    # Добавление расчета метрик обучения для обеих моделей в таблицу
    table.add_row([f"Время выполнения обучения (сек) (эпоха {epoch + 1})", f"{elapsed_time_x3d_xs:.4f}", f"{elapsed_time_slow_r50:.4f}"])
    table.add_row([f"Потери обучения (эпоха {epoch + 1})", f"{loss_x3d_xs.item():.4f}", f"{loss_slow_r50.item():.4f}"])

# Вывод таблицы
print(table)

Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main
Using cache found in /root/.cache/torch/hub/facebookresearch_pytorchvideo_main


+-------------------------------------------+---------+----------+
|                  Метрика                  |  X3D_XS | Slow_R50 |
+-------------------------------------------+---------+----------+
| Время выполнения обучения (сек) (эпоха 1) | 19.1148 | 47.5180  |
|         Потери обучения (эпоха 1)         |  6.0075 |  6.0916  |
| Время выполнения обучения (сек) (эпоха 2) | 19.6121 | 48.4503  |
|         Потери обучения (эпоха 2)         |  5.9359 |  6.0573  |
+-------------------------------------------+---------+----------+
